In [1]:
import os
import numpy as np
import torch
from moleculekit.molecule import Molecule
import parmed
import nglview as ng
import MDAnalysis as md

import openmm.app as app
from openmm import Platform, LangevinMiddleIntegrator, XmlSerializer
from openmm.unit import picosecond, kelvin
from openmm import unit
import openmm

import mymd

In [2]:
data_path = '../../data/acc_md_with_ai/aspirin/'
pdb_file = os.path.join(data_path, 'aspirin.pdb')
prmtop_file = os.path.join(data_path, 'aspirin.prmtop')

cutoff = 5
box = np.array([20., 20., 20.])
T = 300
device = 'cpu'
precision = torch.float
use_centered = True

In [3]:
mol = mymd.get_molecule(prmtop_file=prmtop_file, pdb_file=pdb_file)

2023-05-15 22:10:21,835 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [4]:
class MyAmberParameterSet(parmed.amber.AmberParameterSet):
    """Inherited from AmberParameterSet to modify from_structure method."""

    @classmethod
    def my_from_structure(cls, struct):
        return super(parmed.amber.AmberParameterSet, cls).from_structure(struct, allow_unequal_duplicates=True)


structure = parmed.load_file(prmtop_file)
try:
    prm = parmed.amber.AmberParameterSet().from_structure(structure)
except:
    print('wrong')
    prm = MyAmberParameterSet.my_from_structure(structure)
print(isinstance(structure, parmed.amber.AmberParm))

a = unit.Quantity((box[0] * unit.angstrom, 0 * unit.angstrom, 0 * unit.angstrom))
b = unit.Quantity((0 * unit.angstrom, box[1] * unit.angstrom, 0 * unit.angstrom))
c = unit.Quantity((0 * unit.angstrom, 0 * unit.angstrom, box[2] * unit.angstrom))
structure.box_vectors = (a, b, c)


def disableDispersionCorrection(system):
    # According to openMM:
    # The long range dispersion correction is primarily useful when running simulations at constant pressure, since it
    # produces a more accurate variation in system energy with respect to volume.
    # So I will disable it to avoid implementing it for now.
    from openmm import NonbondedForce

    for f in system.getForces():
        if isinstance(f, NonbondedForce):
            f.setUseDispersionCorrection(False)


system = structure.createSystem(nonbondedMethod=app.CutoffPeriodic,
                                nonbondedCutoff=cutoff * unit.angstrom,
                                switchDistance=0)
system.setDefaultPeriodicBoxVectors(a, b, c)
disableDispersionCorrection(system)

integrator = openmm.LangevinIntegrator(T * unit.kelvin, 1 / unit.picoseconds, 2 * unit.femtoseconds)
platform = openmm.Platform.getPlatformByName('OpenCL')
properties = dict()

context = openmm.Context(system, integrator, platform, properties)
context.setPositions(mol.coords * unit.angstrom)
energies = parmed.openmm.energy_decomposition(structure, context)
energies

2023-05-15 22:10:21,981 - parmed.structure - INFO - Adding bonds...
2023-05-15 22:10:21,982 - parmed.structure - INFO - Adding angles...
2023-05-15 22:10:21,982 - parmed.structure - INFO - Adding dihedrals...
2023-05-15 22:10:21,982 - parmed.structure - INFO - Adding Ryckaert-Bellemans torsions...
2023-05-15 22:10:21,982 - parmed.structure - INFO - Adding Urey-Bradleys...
2023-05-15 22:10:21,983 - parmed.structure - INFO - Adding improper torsions...
2023-05-15 22:10:21,983 - parmed.structure - INFO - Adding CMAP torsions...
2023-05-15 22:10:21,983 - parmed.structure - INFO - Adding trigonal angle terms...
2023-05-15 22:10:21,983 - parmed.structure - INFO - Adding out-of-plane bends...
2023-05-15 22:10:21,983 - parmed.structure - INFO - Adding pi-torsions...
2023-05-15 22:10:21,984 - parmed.structure - INFO - Adding stretch-bends...
2023-05-15 22:10:21,984 - parmed.structure - INFO - Adding torsion-torsions...
2023-05-15 22:10:21,984 - parmed.structure - INFO - Adding Nonbonded force..

True


{'bond': 3.0544656640484957,
 'angle': 1.706726350474312,
 'dihedral': 5.671557687208711,
 'nonbonded': -105.55332978870399,
 'total': -95.12058498754556}

In [5]:
try:
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=False)
except:
    print('False causes error, use True.')
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=True)

system = mymd.System(mol, ff, cutoff=cutoff)
system.set_periodic_box_manual(box.reshape(3, 1))
integrator = mymd.VelocityVerletIntegrator(2)

simulation = mymd.Simulation(mol, system, integrator, device, precision, use_centered=use_centered)
simulation.set_positions(mol.coords)
simulation.set_velocities_to_temperature(T=T)
simulation.update_potentials_and_forces()

print(simulation.potentials)
print(simulation.potentials_sum)
print(torch.max(torch.abs(simulation.forces)))
print(torch.min(torch.abs(simulation.forces)))
print(torch.max(simulation.forces))
print(torch.min(simulation.forces))

{'bonds': 3.0544779300689697, 'angles': 1.7067383527755737, 'dihedrals': 5.644763469696045, 'impropers': 0.026793889701366425, 'lj': 18.31438446044922, 'electrostatics': -73.98635864257812, 'external': 0.0}
-45.23920053988695
tensor(72.1620)
tensor(0.1192)
tensor(67.1751)
tensor(-72.1620)


In [6]:
dataset = np.load(os.path.join(data_path, 'md17_aspirin.npz'))
print(dataset['E'][:15].flatten())
for i in range(5):
    f = dataset['F'][i]
    print(np.max(np.abs(f)), np.min(np.abs(f)), np.max(f), np.min(f))

[-406757.59126402 -406757.03429293 -406755.65420705 -406753.83377324
 -406752.02145762 -406750.54737373 -406749.51483279 -406748.80367635
 -406748.1725602  -406747.38329599 -406746.26498811 -406744.73929505
 -406742.85736811 -406740.84565851 -406739.05751445]
56.17724192114808 0.0026730380257048757 56.17724192114808 -48.17346319307507
51.36705972152318 0.03814276481278168 51.36705972152318 -44.808529875306824
51.0095611534444 0.010307566469301107 46.963941065728065 -51.0095611534444
56.77521034843334 0.32637686012994876 44.01304762566052 -56.77521034843334
61.37912908705054 0.27040312881624945 50.67915124132727 -61.37912908705054
